## Output Parsing

Language models output text. But there are times where you want to get more structured information than just text back

Output parsers are classes that help structure language model responses. There are two main methods an output parser must implement:

- **Get format instructions**: A method which returns a string containing instructions for how the output of a language model should be formatted.
- **Parse**: A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.

- Output Parsing
    - StrOutputParser
    - JsonOutputParser
    - CSV Output Parser
    - Datatime Output Parser
    - Structured Output Parser (Pydanitc or Json)


### `Pydantinc` Output Parser

In [1]:
from dotenv import load_dotenv

load_dotenv('./../../.env')

# langfuse or opik

True

In [18]:
import pandas as pd
df = pd.read_parquet("hf://datasets/AndresR2909/youtube_transcriptions_summaries_gpt4o/data/train-00000-of-00001.parquet")

df.info()

/Users/andrestrepo/Documents/repos_personal/Langchain-and-Ollama/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3256 entries, 0 to 3255
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   chanel_name           3256 non-null   object 
 1   video_id              3256 non-null   object 
 2   source                3256 non-null   object 
 3   publish_date          3141 non-null   object 
 4   duration              3256 non-null   float64
 5   last_update_date      3256 non-null   object 
 6   title                 3256 non-null   object 
 7   text                  3256 non-null   object 
 8   year                  3256 non-null   int64  
 9   number_of_tokenks     3256 non-null   int64  
 10  number_of_new_tokens  942 non-null    float64
 11  prompt                3256 non-null   object 
 12  summary               3256 non-null   object 
 13  key_terms             3256 non-null   object 
 14  __index_level_0__     3256 non-null   int64  
dtypes: float64(2), int64(

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import (
                                        SystemMessagePromptTemplate,
                                        HumanMessagePromptTemplate,
                                        ChatPromptTemplate,
                                        PromptTemplate
                                        )

base_url = "http://localhost:11434"
model = 'phi4:latest'#'llama3.2:3b'

llm = ChatOllama(base_url=base_url, model=model)

In [12]:
from typing import  Optional, List
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser


In [65]:
class ResumenTranscripcion(BaseModel):
    resumen: str = Field(description="Introducción que ofrece una visión general del tema")
    puntos_importantes: List[str] = Field(description="Lista de los puntos más importantes ('Bullet Points')")
    conclusion: str = Field(description="Conclusión del resumen")
    activos_mencionados: Optional[List[str]] = Field(description="Lista de los activos mencionados para invertir", default=None)

In [66]:
parser = PydanticOutputParser(pydantic_object=ResumenTranscripcion)

In [67]:
instruction = parser.get_format_instructions()

In [24]:
print(instruction)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"resumen": {"description": "Introducción que ofrece una visión general del tema", "title": "Resumen", "type": "string"}, "puntos_importantes": {"description": "Lista de los puntos más importantes ('Bullet Points') en formato de viñetas", "items": {"type": "string"}, "title": "Puntos Importantes", "type": "array"}, "conclusion": {"description": "Conclusión del resumen", "title": "Conclusion", "type": "string"}, "activos_mencionados": {"anyOf": [{"items": {"type": "string"}, "type": "array"}, {"type": "null"}], "default": null, "

In [36]:
prompt = PromptTemplate(
    template='''
    Eres un experto de trading y análisis de mercado. Tu trabajo consiste en elaborar un informe con los tópicos más importantes del siguiente texto:
    ------------
    {texto}
    ------------
    Comience el resumen con una "Introducción" que ofrezca una visión general del tema seguido por los puntos más importantes ("Bullet Points"). Termina el resumen con una conclusión.
    **Además, extrae los activos mencionados para invertir y crea una lista separada de los mismos.**
    {format_instruction}
    Respuesta:''',
    input_variables=['texto'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)
chain = prompt | llm

In [37]:
texto_ejemplo = df.text.iloc[0]
print(texto_ejemplo)

bitcoin: long eterno comunitario, todos a la misma crypto | btc | cryptos | etf \ eth \ xrp. vamos vamos vamos bitcoin ópera vamos vamos a ganar sí hoy es en ópera no me importa sub baja tengo que respirar solo debo yo aguantar hola mis amigos cómo les va les saluda greg quiroga bienvenidos una vez más a domingo de iglesia cripto gráfica esto es usa crypto noticias gracias a todos los que se están conectando hoy domingo temprano en la madrugada mis amigos que están allá en la cama haciendo raskin ball sí con una mano el control y con la otra haciendo practicando el deporte y el raskin ball que llaman saludos a todos bienvenidos a esta emisión del día de hoy miren cosas importantes del día de hoy se le está pasando el patrón de caída bitcoin en el gráfico diario y no caen muy mucho importante está haciendo un rango por allá ibiña por allá ibiña un ranguiñ ahora me dicen los brasileros por qué me hables brasilero entonces sí un ranguiñ por allá arriba bitcoin muy importante porque se est

In [38]:
output = chain.invoke({'texto': texto_ejemplo})

In [39]:
print(output.content)

```json
{
  "resumen": "El contenido proporcionado parece ser una discusión en vivo sobre estrategias de inversión, específicamente enfocándose en el trading del activo Solana (SOL). Los participantes comparten ideas y dudas sobre cómo realizar inversiones a largo plazo y el manejo de riesgos asociados con fluctuaciones significativas en los precios.",
  "puntos_importantes": [
    "Bastian menciona que está aprendiendo sobre inversión a largo plazo, indicando una estrategia de 'hodl' para soportar caídas del 70%.",
    "El tema central es la inversión en el activo Solana (SOL), discutiendo el impacto potencial de movimientos significativos en su precio.",
    "Se aconseja realizar inversiones de prueba, como un 'long eterno', para aprender a manejar emociones durante las caídas del mercado.",
    "La conversación también incluye consejos sobre cómo empezar con pequeñas cantidades de dinero y aumentar gradualmente la inversión conforme se gana experiencia.",
    "Se discute el impacto 

In [29]:
chain = prompt | llm | parser
output = chain.invoke({'texto': texto_ejemplo})
print(output)

resumen="El texto proporciona un resumen de una discusión en vivo sobre estrategias de inversión, centrándose principalmente en la implementación y observaciones del 'long eterno' o 'long aislado'. La conversación aborda técnicas para invertir con poco capital, el comportamiento histórico de los activos y consideraciones estratégicas." puntos_importantes=['La importancia de iniciar un long aislado con pequeñas cantidades de dinero se destaca como una estrategia para minimizar riesgos.', "El 'long eterno' es una estrategia que busca aprovechar tendencias alcistas prolongadas, comenzando en niveles bajos y manteniendo la posición durante largos períodos.", 'Los participantes discuten el comportamiento de activos específicos como Solana (SOL) y Bitcoin, analizando su potencial de crecimiento y capitalización de mercado.', 'Se recomienda no intentar alcanzar picos muy altos de precios inmediatamente, sino centrarse en objetivos más realistas a corto plazo para lograr ganancias.', 'Los part

### Parsing with `.with_structured_output()` method
- This method takes a schema as input which specifies the names, types, and descriptions of the desired output attributes.
-  The schema can be specified as a TypedDict class, JSON Schema or a Pydantic class.


In [62]:
model = 'llama3.2:3b'

llm = ChatOllama(base_url=base_url, model=model)
structured_llm = llm.with_structured_output(ResumenTranscripcion)

In [58]:
prompt ='''Eres un experto de trading y análisis de mercado. Tu trabajo consiste en elaborar un informe con los tópicos más importantes del siguiente texto:
    ------------
    {texto}
    ------------
    Comience el resumen con una "Introducción" que ofrezca una visión general del tema seguido por los puntos más importantes ("Bullet Points"). Termina el resumen con una conclusión.
    **Además, extrae los activos mencionados para invertir y crea una lista separada de los mismos.**
    Respuesta:'''

template = ChatPromptTemplate.from_template(prompt)

In [60]:
query =template.invoke({"texto":texto_ejemplo})
query

ChatPromptValue(messages=[HumanMessage(content='Eres un experto de trading y análisis de mercado. Tu trabajo consiste en elaborar un informe con los tópicos más importantes del siguiente texto:\n    ------------\n    bitcoin: long eterno comunitario, todos a la misma crypto | btc | cryptos | etf \\ eth \\ xrp. vamos vamos vamos bitcoin ópera vamos vamos a ganar sí hoy es en ópera no me importa sub baja tengo que respirar solo debo yo aguantar hola mis amigos cómo les va les saluda greg quiroga bienvenidos una vez más a domingo de iglesia cripto gráfica esto es usa crypto noticias gracias a todos los que se están conectando hoy domingo temprano en la madrugada mis amigos que están allá en la cama haciendo raskin ball sí con una mano el control y con la otra haciendo practicando el deporte y el raskin ball que llaman saludos a todos bienvenidos a esta emisión del día de hoy miren cosas importantes del día de hoy se le está pasando el patrón de caída bitcoin en el gráfico diario y no caen

In [63]:
chain = template | structured_llm

In [64]:
output = chain.invoke({"texto":texto_ejemplo})
print(output)

None


### `JSON` Output Parser

- Output parsers accept a string or BaseMessage as input and can return an arbitrary type.



In [68]:
from langchain_core.output_parsers import JsonOutputParser

In [69]:
parser = JsonOutputParser(pydantic_object=ResumenTranscripcion)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"resumen": {"description": "Introducción que ofrece una visión general del tema", "title": "Resumen", "type": "string"}, "puntos_importantes": {"description": "Lista de los puntos más importantes ('Bullet Points')", "items": {"type": "string"}, "title": "Puntos Importantes", "type": "array"}, "conclusion": {"description": "Conclusión del resumen", "title": "Conclusion", "type": "string"}, "activos_mencionados": {"anyOf": [{"items": {"type": "string"}, "type": "array"}, {"type": "null"}], "default": null, "description": "Lista d

In [75]:
prompt = PromptTemplate(
    template='''Eres un experto de trading y análisis de mercado. Tu trabajo consiste en elaborar un informe con los tópicos más importantes del siguiente texto:
    ------------
    {texto}
    ------------
    Comience el resumen con una "Introducción" que ofrezca una visión general del tema seguido por los puntos más importantes ("Bullet Points"). Termina el resumen con una conclusión.
    **Además, extrae los activos mencionados para invertir y crea una lista separada de los mismos.**
    Aqui estan tus intrucciones de formato de salida:
    {format_instruction}
    Respuesta:''',
    input_variables=['texto'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

In [76]:
prompt_v2 = PromptTemplate(
    template='''
    Eres un experto de trading y análisis de mercado. Tu trabajo consiste en elaborar un informe con los tópicos más importantes del siguiente texto.
    Comience el resumen con una "Introducción" que ofrezca una visión general del tema seguido por los puntos más importantes ("Bullet Points"). Termina el resumen con una conclusión.
    **Además, extrae los activos mencionados para invertir y crea una lista separada de los mismos.**
    Aqui estan tus intrucciones de formato de salida:
    {format_instruction}
    ------------
    texto: {texto}
    ------------
    Respuesta:''',
    input_variables=['texto'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

In [77]:
model = 'phi4:latest'#'llama3.2:3b'

llm = ChatOllama(base_url=base_url, model=model)

In [78]:

chain = prompt | llm
output = chain.invoke({"texto":texto_ejemplo})
print(output.content)

```json
{
  "resumen": "El discurso se centra en la discusión de inversiones financieras, específicamente el manejo y estrategias para operar con criptomonedas como Solana. Se resalta la importancia del análisis técnico, tales como el uso del histograma MACD, y se proporcionan consejos sobre cómo abordar las tendencias del mercado en momentos de alta volatilidad.",
  "puntos_importantes": [
    "Básicamente, hay una discusión sobre estrategias para invertir en criptomonedas a través del uso de operaciones aisladas y posiciones de 'long'.",
    "Se resalta la importancia de usar herramientas como el histograma MACD para analizar tendencias en los mercados.",
    "El discurso también aborda la volatilidad y cómo ciertos activos pueden experimentar fluctuaciones rápidas de precios, destacando la necesidad de estar preparado para tales situaciones.",
    "Se menciona que invertir en criptomonedas puede ofrecer oportunidades significativas pero conlleva un alto riesgo debido a su naturaleza

In [79]:
chain = prompt | llm | parser
output = chain.invoke({"texto":texto_ejemplo})
print(output)

{'resumen': 'El resumen proporciona una discusión detallada sobre estrategias de inversión, con un enfoque en el análisis técnico y la gestión del riesgo. Varios participantes comparten sus experiencias e ideas sobre cómo invertir efectivamente utilizando diferentes activos.', 'puntos_importantes': ['La importancia de iniciar una posición larga (long) aislada con un pequeño capital, sugiriendo usar 3x para mitigar riesgos.', 'Discusión en torno al análisis técnico y cómo interpretar indicadores como el MACD y el histograma, enfatizando la necesidad de no leer los indicadores de manera literal.', 'Mencionada una proyección positiva para un activo llamado Sol, resaltando su capitalización de mercado y potencial de crecimiento.', 'La comunidad intercambia tácticas sobre cuándo invertir o cortar posiciones, subrayando la importancia del seguimiento constante y adaptativo.', 'Se discute la incertidumbre en los movimientos del mercado debido a fuerzas externas impredecibles, sugiriendo estra

### CSV Output Parser

- This output parser can be used when you want to return a list of comma-separated items.



In [80]:
# value1, values2, values3, so on

from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()

print(parser.get_format_instructions())

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [81]:
format_instruction = parser.get_format_instructions()

prompt = PromptTemplate(
    template='''
    Answer the user query with a list of values. Here is your formatting instruction.
    {format_instruction}

    Query: {query}
    Answer:''',
    input_variables=['query'],
    partial_variables={'format_instruction': format_instruction}
)   

In [82]:
chain = prompt | llm | parser

output = chain.invoke({'query': 'generate my website seo keywords. I have content about the NLP and LLM.'})
print(output)

['Natural Language Processing', 'Large Language Models', 'NLP Techniques', 'LLM Applications', 'AI in Linguistics', 'Machine Learning for Text Analysis', 'Conversational Agents', 'Semantic Analysis', 'Text Summarization', 'Sentiment Analysis', 'Speech Recognition', 'Language Translation', 'Chatbots', 'Natural Language Understanding', 'Dialogue Systems', 'Transformer Models', 'BERT', 'GPT', 'Contextual Embeddings', 'NLP Tools', 'LLM Frameworks', 'AI-driven Communication', 'Text Classification', 'Named Entity Recognition', 'Machine Learning Algorithms for NLP', 'Deep Learning in NLP', 'Language Model Training', 'Data Annotation for NLP', 'NLP Libraries and APIs', 'LLM Research', 'Natural Language Generation', 'Question Answering Systems', 'Semantic Parsing', 'Syntax Analysis', 'Computational Linguistics', 'Predictive Text Modeling']


### Datatime Output Parser

- Gives output in datetime format. Sometimes throws error if the LLM output is not in datetime format.

In [83]:
from langchain.output_parsers import DatetimeOutputParser

In [84]:
parser = DatetimeOutputParser()

format_instruction = parser.get_format_instructions()
print(format_instruction)

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 0719-03-29T07:27:22.420670Z, 0412-10-14T02:43:44.992807Z, 1268-07-28T08:02:00.620018Z

Return ONLY this string, no other words!


In [85]:
prompt = PromptTemplate(
    template='''
    Answer the user query with a datetime. Here is your formatting instruction.
    {format_instruction}

    Query: {query}
    Answer:''',
    input_variables=['query'],
    partial_variables={'format_instruction': format_instruction}
)

In [86]:
chain = prompt | llm | parser

In [90]:
output = chain.invoke({'query': 'when the America got discovered?'})

print(output)

1492-10-12 00:00:00
